In [ ]:
import nba_api
from nba_api.stats.endpoints import leaguegamefinder



In [ ]:
from nba_api.stats.static import players
from nba_api.stats.endpoints import playergamelog

# Get player dictionary
player_dict = players.get_players()

# Find player by name
lebron = [player for player in player_dict if player['full_name'] == 'LeBron James'][0]

# Query player's game log (for a specific season, e.g., 2021-22)
gamelog = playergamelog.PlayerGameLog(player_id=lebron['id'], season='2023')
df = gamelog.get_data_frames()[0]

print(df.head())


In [2]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import teamgamelog
import pandas as pd

def get_team_id(team_name):
    """
    Returns the team ID for a given team name.
    """
    nba_teams = teams.get_teams()
    for team in nba_teams:
        if team['full_name'].lower() == team_name.lower():
            return team['id']
    return None

def average_points_per_game(team_name, season):
    """
    Prints the average points per game for a given team and season.
    """
    team_id = get_team_id(team_name)
    if team_id is None:
        print("Team not found.")
        return
    
    game_log = teamgamelog.TeamGameLog(team_id=team_id, season=season)
    game_log_df = game_log.get_data_frames()[0]
    
    # Calculate average points
    average_points = game_log_df['PTS'].mean()
    print(f"{team_name} average points per game in {season}: {average_points:.2f}")

# Example usage
team_name = input("Enter team name (e.g., 'Los Angeles Lakers'): ")
season = input("Enter season (format YYYY-YY, e.g., '2021-22'): ")

average_points_per_game(team_name, season)


Houston Rockets average points per game in 2023-24: 112.83


In [ ]:
def average_points_against_team(team_name, opponent_team_name, season):
    """
    Calculates and prints the average points per game for a given team against another team in a specified season.
    """
    team_id = get_team_id(team_name)
    opponent_team_id = get_team_id(opponent_team_name)
    
    if team_id is None or opponent_team_id is None:
        print("One of the teams was not found.")
        return
    
    # Find games between the two teams
    game_finder = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_id, season_nullable=season)
    games_df = game_finder.get_data_frames()[0]
    
    # Ensure correct team abbreviations are used
    team_abbreviation = [team['abbreviation'] for team in teams.get_teams() if team['id'] == team_id][0]
    opponent_team_abbreviation = [team['abbreviation'] for team in teams.get_teams() if team['id'] == opponent_team_id][0]
    
    # Filter games against the opponent team
    games_against_opponent = games_df[games_df['MATCHUP'].str.contains(opponent_team_abbreviation)]
    
    # Calculate average points
    if not games_against_opponent.empty:
        average_points = games_against_opponent['PTS'].mean()
        print(f"Average points scored by {team_name} against {opponent_team_name} in {season}: {average_points:.2f}")
    else:
        print(f"No games found between {team_name} and {opponent_team_name} in {season}.")

# Example usage
team_name = "Boston Celtics"
opponent_team_name = "Houston Rockets"
season = "2023-24"

average_points_against_team(team_name, opponent_team_name, season)
average_points_against_team(opponent_team_name, team_name, season)


## Team Performance Metrics 

In [ ]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import teamyearbyyearstats, leaguedashteamstats

import pandas as pd


In [ ]:
def get_team_performance_metrics(team_name, season):
    # Fetching team stats for the given season
    team_stats = leaguedashteamstats.LeagueDashTeamStats(season=season)
    team_stats_df = team_stats.get_data_frames()[0]
    
    # Filter for the given team name
    team_stats_row = team_stats_df[team_stats_df['TEAM_NAME'].str.lower() == team_name.lower()]
    
    if team_stats_row.empty:
        print(f"No data found for {team_name} in {season}.")
        return

    games_played = team_stats_row['GP'].values[0]
    total_points = team_stats_row['PTS'].values[0]
    total_rebounds = team_stats_row['REB'].values[0]
    total_assists = team_stats_row['AST'].values[0]
    total_turnovers = team_stats_row['TOV'].values[0]
    total_steals = team_stats_row['STL'].values[0]
    total_blocks = team_stats_row['BLK'].values[0]
    total_points_allowed = team_stats_row['PTS_AGAINST'].values[0] if 'PTS_AGAINST' in team_stats_row.columns else None

    # Calculate the average stats per game
    ppg = total_points / games_played if games_played else 0
    rpg = total_rebounds / games_played if games_played else 0
    apg = total_assists / games_played if games_played else 0
    topg = total_turnovers / games_played if games_played else 0
    spg = total_steals / games_played if games_played else 0
    bpg = total_blocks / games_played if games_played else 0
    papg = total_points_allowed / games_played if total_points_allowed and games_played else 0

    metrics_dict = {
        'Points Per Game (PPG)': ppg,
        'Field Goal Percentage (FG%)': team_stats_row['FG_PCT'].values[0],
        '3-Point Percentage (3P%)': team_stats_row['FG3_PCT'].values[0],
        'Free Throw Percentage (FT%)': team_stats_row['FT_PCT'].values[0],
        'Rebounds Per Game (RPG)': rpg,
        'Assists Per Game (APG)': apg,
        'Turnovers Per Game (TOPG)': topg,
        'Steals Per Game (SPG)': spg,
        'Blocks Per Game (BPG)': bpg,
        'Points Allowed Per Game (PAPG)': papg,
    }

    return metrics_dict

# Example usage
team_name = input("Enter the team name (e.g., 'Los Angeles Lakers'): ")
season = input("Enter the season (format YYYY-YY, e.g., '2023-24'): ")
metrics = get_team_performance_metrics(team_name, season)
print(f"{team_name}' Performance metrics:")
if metrics:
    for metric, value in metrics.items():
        print(f"{metric}: {value:.2f}")


To calculate advanced metrics such as Efficiency, Win Shares, Player Impact Estimate (PIE), Offensive and Defensive Ratings, and Pace, you will typically need detailed play-by-play data and box score statistics. Some of these metrics can be complex, as they require a combination of multiple statistical elements and sometimes proprietary formulas.

Here's a brief overview of how you might approach each one:

Efficiency (Team Efficiency Rating)
Team Efficiency Rating is not an official NBA statistic and can vary depending on the formula you use. A simple version might be points scored minus points allowed, divided by possessions.

Win Shares
Win Shares is a player statistic that attempts to divvy up credit for team success to the individuals on the team. It's calculated using player, team, and league statistics and is quite complex. You may need to rely on a data provider to supply win shares unless you want to implement the full calculation yourself.

Player Impact Estimate (PIE)
PIE measures a player's overall statistical contribution against the total statistics in games they play in. The formula for PIE is as follows:

plaintext
Copy code
PIE = (Player Stats) / (Team Stats + Opponent Stats)
Each "Player Stats" and "Team Stats" category includes points, field goals made, free throws made, defensive rebounds, offensive rebounds, assists, steals, blocks, and turnovers.

Offensive and Defensive Ratings
These are estimates of a team's performance per 100 possessions.

Offensive Rating (ORtg) is the number of points a team scores per 100 possessions.
Defensive Rating (DRtg) is the number of points a team allows per 100 possessions.
To calculate these, you typically need box score data and possession estimates.

Pace
Pace factor is an estimate of the number of possessions per 48 minutes by a team. A commonly used formula for pace is:

plaintext
Copy code
Pace = 48 * ((Tm Poss + Opp Poss) / (2 * (Tm MP / 5)))
Where:

Tm Poss is an estimate of the team's possessions
Opp Poss is an estimate of the opponent's possessions
Tm MP is the minutes played, which is typically 240 for a regulation game
To get these metrics using nba_api, you can use endpoints such as teamgamelog for game-by-game statistics, boxscoreadvancedv2 for advanced box scores, and leaguegamelog for league-wide game logs. However, due to the complexity and proprietary nature of some of these statistics, you may also want to consider paid data services that provide these metrics pre-calculated.

In [6]:
from nba_api.stats.endpoints import teamdashboardbygeneralsplits

def get_advanced_team_metrics(team_name, season):
    # Get the team id
    team_id = get_team_id(team_name)
    
    # Fetch the advanced team stats for the given season
    team_advanced_stats = teamdashboardbygeneralsplits.TeamDashboardByGeneralSplits(team_id=team_id, season=season, measure_type_detailed_defense='Advanced')
    team_advanced_stats_df = team_advanced_stats.get_data_frames()[0]
    
    # Extract the relevant metrics
    metrics_dict = {
        'Offensive Rating': team_advanced_stats_df['OFF_RATING'].values[0],
        'Defensive Rating': team_advanced_stats_df['DEF_RATING'].values[0],
        'Net Rating': team_advanced_stats_df['NET_RATING'].values[0],
        'Pace': team_advanced_stats_df['PACE'].values[0],
        'Player Impact Estimate': team_advanced_stats_df['PIE'].values[0]
    }

    return metrics_dict

# Example usage
# team_name = "Houston Rockets"  # For example
# season = "2023-24"  # For example
advanced_metrics = get_advanced_team_metrics(team_name, season)
print(advanced_metrics)
print(f"{team_name}' Advanced metrics:")
if advanced_metrics:
    for metric, value in advanced_metrics.items():
        print(f"{metric}: {value:.2f}")


{'Offensive Rating': 112.4, 'Defensive Rating': 112.8, 'Net Rating': -0.4, 'Pace': 99.43, 'Player Impact Estimate': 0.496}
Houston Rockets' Performance metrics:
Offensive Rating: 112.40
Defensive Rating: 112.80
Net Rating: -0.40
Pace: 99.43
Player Impact Estimate: 0.50
